In [4]:
# 라이브러리 불러오기
import threading
import cv2
import math
import numpy as np
import matplotlib.pyplot as plt

from time import sleep
# from yolov3.detect_ceustom import run_yolo
from pose_ArUCo.pose_estimation import run_aruco
from pose_ArUCo.detect_aruco_images import detect_marker
from dobot_api import DobotApiDashboard, DobotApi, DobotApiMove, MyType
from rotate import transform_cam_to_robot

In [ ]:
# yolo 물체 검출 결과 얻어오기
# 입력 : None
# 출력 : 검출된 물체의 (색상, 모양, 검출 박스의 중앙 x, y 좌표)
def get_yolo_results():
    # 물체 정보
    c_dict = {'0': 'red', '1': 'yellow', '2': 'green'}               # 색상
    s_dict = {'0': 'Hexagonal_Prism', '1': 'Cylinder', '2': 'Cube'}  # 모양

    # YOLO 기반 물체 검출 
    result, imc = run_yolo()    # result : YOLO 물체 검출 결과 정보
    result_list = []            # imc : 원본 이미지 (aruco 마커 검출용)

    # 물체 검출 결과 전처리
    for boxes in result:
        w = float(boxes.split()[-2])
        h = float(boxes.split()[-1])

        for i, v in enumerate(boxes.split()):
            # 색상, 모양 정보
            if i == 0:
                color = c_dict[v]
                shape = s_dict[v]

            # 검출된 박스의 중앙 x 좌표 (0~1 정규화 값 -> 이미지 픽셀 값으로 보정)
            elif i == 1:
                x = int(float(v) * w)

            # 검출된 박스의 중앙 y 좌표 (0~1 정규화 값 -> 이미지 픽셀 값으로 보정)
            elif i == 2:
                y = int(float(v) * h)

        result_list.append([color, shape, x, y])

    return result_list

In [ ]:
# 전역 변수 (현재 좌표)
current_actual = None

# PC-로봇 연결 함수 (TCP/IP 통신)
def connect_robot(ip):
    try:
        dashboard_p = 29999
        move_p = 30003
        feed_p = 30004
        print("연결 설정 중...")

        dashboard = DobotApiDashboard(ip, dashboard_p)
        move = DobotApiMove(ip, move_p)
        feed = DobotApi(ip, feed_p)
        print("연결 성공!!")

        return dashboard, move, feed

    except Exception as e:
        print(":연결 실패ㅜㅜ")
        raise e

In [ ]:
# 로봇 에러 메시지 초기화 함수
def robot_clear(dashboard : DobotApiDashboard):
    dashboard.ClearError()

# 로봇 속도 조절 함수 (speed_value : 1~100)
def robot_speed(dashboard : DobotApiDashboard, speed_value):
    dashboard.SpeedFactor(speed_value)

# 그리퍼 구동 함수 (status 0 -> OFF, 1-> ON)
def gripper_DO(dashboard : DobotApiDashboard, index, status):
    dashboard.ToolDO(index, status)

# 현재 로봇 위치 받아오기 (로봇 베이스 좌표계 기준)
def get_Pose(dashboard : DobotApiDashboard):
    dashboard.GetPose()

In [ ]:
# 로봇 구동 함수 (현재 위치 -> 목표 위치(point_list))
def run_point(move: DobotApiMove, point_list: list):
    move.MovL(point_list[0], point_list[1], point_list[2], point_list[3])
    
def get_feed(feed: DobotApi):
    global current_actual
    hasRead = 0

    while True:
        data = bytes()
        while hasRead < 1440:
            temp = feed.socket_dobot.recv(1440 - hasRead)
            if len(temp) > 0:
                hasRead += len(temp)
                data += temp
        hasRead = 0
        a = np.frombuffer(data, dtype=MyType)

        if hex((a['test_value'][0])) == '0x123456789abcdef':
            current_actual = a["tool_vector_actual"][0]     # Refresh Properties
        sleep(0.001)

In [ ]:
### 입력 파라미터 #################################################################################
ip = "192.168.1.6"                  # Robot의 IP 주소
speed_value = 10                    # 로봇 속도 (1~100 사이의 값 입력)
gripper_port = 1                    # 그리퍼 포트 번호
point_home = [150, 200, 100, 180]   # 로봇의 초기 좌표 (x, y, z, yaw)
point_goal = [245, -7, 55, 0]        

goal = "Hexagonal_Prism"            # 목표 물체 유형 (Hexagonal_Prism | Cylinder | Cube)
target_point = []                   # 목표 물체에 대한 박스 정보 (중앙 x, y 좌표)
goal_index = 0
goal_point = [0, 0, 0]              # 목표 물체에 대한 x, y, z 좌표 (카메라 좌표계 기준)
###################################################################################################

In [ ]:
# YOLO 기반 물체 검출 -> 물체 색상, 모양, 박스의 중앙 좌표 얻어오기
result_list = get_yolo_results()

print(f"result_list : {result_list}")

In [ ]:
# 웹캠 이미지 받아오기
cap = cv2.VideoCapture(0)
ret, img = cap.read()

# 웹캠 이미지를 RGB로 변환
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

In [ ]:
# 아르코 마커로 카메라 좌표계 기준 6D pose 예측하기
# rvec : 회전 행렬 (roll, pitch, yaw)   // unit : rads
# tvec : 이동 행렬 (x, y, z)            // unit : meter
# corners : 아르코 마커의 각 코너 좌표  // (x, y) 픽셀 위치
rvec, tvec, corners = run_aruco(img_rgb, "DICT4X4_100")

# 간단한 전처리
rvec = np.array(rvec)[0][0]
tvec = np.array(tvec)[0][0]
corners = np.array(corners[0][0])

In [ ]:
# yolo 결과물 중 목표 물체의 중앙 좌표만 추출하기
for i in range(0, len(result_list)):
    if result_list[i][1] == goal:
        print(f"Finding Traget : {result_list[i][1]} !!")
        goal_index = i
        target_point.append([result_list[i][2], result_list[i][3]])
        
print(f"Target point of YOLO : {target_point}")
print(f"goal_index : {goal_index}")

In [ ]:
# 타겟 물체의 이동 및 회전 행렬 받아오기
goal_pose = tvec[goal_index]
goal_angle = rvec[goal_index]

In [ ]:
# 타겟 물체의 좌표 및 각도 출력 (카메라 좌표계 기준)
goal_pose[0] = np.round(goal_pose[0], 4)
goal_pose[1] = np.round(goal_pose[1], 4)
goal_pose[2] = np.round(goal_pose[2], 4)
yaw_angle_by_cam = np.round(goal_angle[-1], 4)

print(f"x based camera cordinate: {goal_pose[0]} meter")
print(f"y based camera cordinate: {goal_pose[1]} meter")
print(f"z based camera cordinate: {goal_pose[2]} meter")
print(f"yaw_angle_by_cam : {yaw_angle_by_cam} rads")

In [ ]:
### 오차 값 보정 ##################################################################################
edit_x = -49.4    # unit : mm
edit_y = -25.9    # unit : mm
###################################################################################################

# 카메라 좌표계 -> 로봇 베이스 좌표계로 변환
point_grip = transform_cam_to_robot(goal_pose)

# 오차 보정 (로봇 베이스 좌표계 기준)
point_grip[0] = round(point_grip[0] + edit_x, 4)
point_grip[1] = round(point_grip[1] + edit_y, 4)

# 고정 값
yaw_angle_by_robot = 15
point_grip[2] = -55.5

# yaw 변환 각도 값 추가
point_grip = np.append(point_grip, yaw_angle_by_robot)
print(f"x pose by robot : {point_grip[0]} mm")
print(f"y pose by robot : {point_grip[1]} mm")
print(f"z pose by robot : {point_grip[2]} mm")
print(f"yaw angle by robot : {point_grip[3]} degree")

In [ ]:
# 로봇 연결
dashboard, move, feed = connect_robot(ip)
dashboard.EnableRobot()
print("이제 로봇을 사용할 수 있습니다!")

# 쓰레드 설정
feed_thread = threading.Thread(target=get_feed, args=(feed,))
feed_thread.setDaemon(True)
feed_thread.start()

In [ ]:
# 로봇 상태 초기화 1 : 로봇 에러 메시지 초기화
robot_clear(dashboard) 

# 로봇 상태 초기화 2 : 로봇 속도 조절
robot_speed(dashboard, speed_value)

# 로봇 현재 위치 받아오기 (x, y, z, yaw) - 로봇 베이스 좌표계 
get_Pose(dashboard)

In [ ]:
# 로봇 구동 1 (초기 위치)
run_point(move, point_home)
sleep(5)

# 로봇 구동 2 (그립할 위치)
run_point(move, point_grip)
sleep(5)

# 그리퍼 구동
gripper_DO(dashboard, gripper_port, 1)
sleep(0.1)

# 로봇 구동 3 (목표 위치)
run_point(move, point_goal)
sleep(5)

# 그리퍼 끄기
gripper_DO(dashboard, gripper_port, 0)
sleep(0.1)

# 로봇 구동 1 (초기 위치)
run_point(move, point_home)
sleep(5)

In [ ]:
# 로봇 끄기
dashboard.DisableRobot()

In [ ]:
1일때
[[     3.1212   -0.065163     0.11242]] [[   0.088926   -0.066835     0.63696]]


0일때

[[     3.1212   -0.065163     0.11242]] [[   0.088926   -0.066835     0.63696]]